In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 35663896
paper_name = 'grosjean_blaudez_2022' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/Table_2_Genome-Wide Mutant Screening in Yeast Reveals that the Cell Wall is a First Shield to Discriminate Light From Heavy Lanthanides.XLSX', 
                              sheet_name='Table S1', skiprows=1)

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 630 x 14


In [7]:
original_data.head()

,SGD ID,ORF,La,Yb,ORF name,ORF name description,Description,ORF type,Aliases,H. sapiens homologs,H. sapiens homologs name description,H. sapiens disease,La specific,Yb specific
0,S000006050,YPL129W,MS,-,TAF14,TATA binding protein-Associated Factor,"Subunit of TFIID, TFIIF, INO80, SWI/SNF, and N...",Verified,TATA-binding protein-associated factor TAF14|T...,"MLLT1, MLLT3, YEATS4",myeloid/lymphoid or mixed-lineage leukemia; tr...,-----,-----,-----
1,S000003589,YJL053W,MS,-,PEP8,carboxyPEPtidase Y-deficient,Vacuolar protein component of the retromer; fo...,Verified,retromer subunit PEP8|VPT4|VPS26|GRD6,"VPS26A, VPS26B","VPS26 retromer complex component A, VPS26 retr...",-----,-----,-----
2,S000005000,YNL055C,LS,-,POR1,PORin,Mitochondrial porin (voltage-dependent anion c...,Verified,porin POR1|VDAC|YVDAC1|OMP2,"VDAC1, VDAC2, VDAC3","voltage-dependent anion channel 1, voltage-dep...",-----,-----,-----
3,S000006377,YPR173C,HR,LR,VPS4,Vacuolar Protein Sorting,AAA-ATPase involved in multivesicular body (MV...,Verified,AAA family ATPase VPS4|DID6|VPT10|VPL4|GRD13|E...,"KATNA1, KATNAL1, KATNAL2, VPS4A, VPS4B","katanin p60 (ATPase containing) subunit A 1, k...",-----,-----,-----
4,S000000690,YCR094W,MS,MS,CDC50,Cell Division Cycle,Endosomal protein that interacts with phosphol...,Verified,aminophospholipid translocase regulatory prote...,"TMEM30A, TMEM30B, TMEM30C","transmembrane protein 30A, transmembrane prote...",-----,-----,-----


In [8]:
original_data['orf'] = original_data['ORF'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [10]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [SGD ID, ORF, La, Yb, ORF name, ORF name description, Description, ORF type, Aliases, H. sapiens homologs, H. sapiens homologs name description, H. sapiens disease, La specific, Yb specific, orf]
Index: []


In [12]:
original_data['La'].unique()

array(['MS', 'LS', 'HR', 'MR', '-', 'LR', 'HS'], dtype=object)

In [13]:
original_data['Yb'].unique()

array(['-', 'LR', 'MS', 'MR', 'LS', 'HS', 'HR'], dtype=object)

In [14]:
data_dict = {'HS': -3, 'MS': -2, 'LS': -1, '-': 0, 'LR': 1, 'MR': 2, 'HR': 3}

In [15]:
original_data['La2'] = original_data['La'].apply(lambda x: data_dict[x])
original_data['Yb2'] = original_data['Yb'].apply(lambda x: data_dict[x])

In [17]:
original_data.set_index('orf', inplace=True)

In [18]:
original_data = original_data[['La2','Yb2']].copy()

In [19]:
original_data = original_data.groupby(original_data.index).mean()

In [20]:
original_data.shape

(629, 2)

# Prepare the final dataset

In [21]:
data = original_data.copy()

In [22]:
dataset_ids = [22225, 22226]
datasets = datasets.reindex(index=dataset_ids)

In [23]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [24]:
data.head()

dataset_id,22225,22226
data_type,value,value
orf,,
YAL016W,-1.0,0.0
YAL020C,0.0,1.0
YAL026C,-2.0,0.0
YAL053W,-2.0,1.0
YAL058W,2.0,1.0


## Subset to the genes currently in SGD

In [25]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [26]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,22225,22226
,data_type,value,value
gene_id,orf,,
14,YAL016W,-1.0,0.0
18,YAL020C,0.0,1.0
24,YAL026C,-2.0,0.0
49,YAL053W,-2.0,1.0
53,YAL058W,2.0,1.0


# Normalize

In [27]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [28]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [29]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,22225,22226,22225,22226
,data_type,value,value,valuez,valuez
gene_id,orf,,,,
14,YAL016W,-1.0,0.0,-1.620299,0.071027
18,YAL020C,0.0,1.0,-0.047656,2.414925
24,YAL026C,-2.0,0.0,-3.192942,0.071027
49,YAL053W,-2.0,1.0,-3.192942,2.414925
53,YAL058W,2.0,1.0,3.097630,2.414925


# Print out

In [30]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [32]:
# from IO.save_data_to_db3 import *

In [34]:
# save_data_to_db(data_all, paper_pmid)